In [1]:
import numpy as np
import torch
from scripts.data_prep import create_dataset

In [2]:
input_path = "../data/UT_HAR_OG/input"
annotation_path = "../data/UT_HAR_OG/annotation"

In [3]:
data, labels = create_dataset(input_path, annotation_path)

In [9]:
# PHASE CALIBRATION
def calibrate_phase(data):
    proc_data = torch.zeros_like(data)
    num_antennas = 3
    num_subcarriers = 30

    for idx, tensor in enumerate(data):
        phase = tensor[:,num_antennas*num_subcarriers:]
        difference = torch.zeros_like(tensor[:,0])
        calibrated_phase = phase.clone()

        for i in range(num_subcarriers*num_antennas):
            if i%num_subcarriers == 0:
                  continue
            temp = phase[:,i] - phase[:,i-1]
            difference = difference + np.sign(temp) * (np.abs(temp) > np.pi)
            calibrated_phase[:,i] = phase[:,i] - difference * 2 * np.pi

        for i in range(num_antennas):
              k = (calibrated_phase[:,i*num_subcarriers +num_subcarriers-1] - calibrated_phase[:,i*num_subcarriers]) / (num_subcarriers-1)
              b = torch.mean(calibrated_phase[:,i*num_subcarriers:i*num_subcarriers+num_subcarriers], dim=1)

              for j in range(num_subcarriers):
                    calibrated_phase[:,i*num_subcarriers+j] -= k*j+b 

   

        
        proc_data[idx, :, :num_antennas*num_subcarriers] = tensor[:,:num_antennas*num_subcarriers]
        proc_data[idx, :, num_antennas*num_subcarriers:] = calibrated_phase

    return proc_data

In [10]:
test_output = calibrate_phase(data)

/tmp/ipykernel_991494/3841141425.py:16: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  difference = difference + np.sign(temp) * (np.abs(temp) > np.pi)


In [12]:
test_output.shape

torch.Size([110793, 10, 180])

In [31]:
print(test_data[0,1,90:])

tensor([13.0541, 12.8528, 12.5417, 12.3269, 12.2419, 12.1556, 12.2698, 12.3829,
        12.2949, 12.4182, 12.5258, 12.5947, 12.7153, 12.7861, 13.2515, 13.1657,
        13.2450, 13.3139, 13.4153, 13.4706, 13.4897, 13.5325, 13.6064, 13.5721,
        13.5681, 13.4418, 13.2470, 13.0113, 12.7521, 13.0541, 11.6246, 11.2058,
        10.7448, 10.4721, 10.1550, 10.1911, 10.4981, 11.1501, 12.3659, 12.6543,
        12.5758, 12.4545, 12.4917, 12.5596, 12.8798, 12.6156, 12.6067, 12.4535,
        12.3296, 12.2669, 12.2013, 12.0815, 12.0784, 11.9979, 11.9683, 11.8456,
        11.6292, 11.5292, 11.3393, 11.6246, 12.7560, 12.4817, 12.2645, 12.1518,
        12.0226, 11.9258, 11.9277, 11.9358, 12.0245, 12.0335, 12.0432, 12.1066,
        12.2620, 12.3237, 12.8686, 12.8882, 12.9916, 13.0743, 13.1368, 13.1890,
        13.2146, 13.1859, 13.2316, 13.1954, 13.2015, 13.0768, 12.8807, 12.7291,
        12.4604, 12.7560])


In [6]:
proc_data = torch.zeros_like(test_data)
num_antennas = 3
num_subcarriers = 30

In [18]:
for idx, tensor in enumerate(test_data):
        phase = tensor[:,num_antennas*num_subcarriers:]
        difference = torch.zeros_like(tensor[:,0])
        calibrated_phase = phase.clone()

        for i in range(num_subcarriers*num_antennas):
            if i%num_subcarriers == 0:
                  continue
            temp = phase[:,i] - phase[:,i-1]
            difference = difference + np.sign(temp) * (np.abs(temp) > np.pi)
            calibrated_phase[:,i] = phase[:,i] - difference * 2 * np.pi

        for i in range(num_antennas):
              k = (calibrated_phase[:,i*num_subcarriers +num_subcarriers-1] - calibrated_phase[:,i*num_subcarriers]) / (num_subcarriers-1)
              b = torch.mean(calibrated_phase[:,i*num_subcarriers:i*num_subcarriers+num_subcarriers], dim=1)

              for j in range(num_subcarriers):
                    calibrated_phase[:,i*num_subcarriers+j] -= k*j+b 

        print(phase[0,30], phase[0,29])
        print(calibrated_phase[0])

tensor(12.2703) tensor(12.2703)
tensor([ 0.0764, -0.1050, -0.3963, -0.5860, -0.7604, -0.8082, -0.7323, -0.6377,
        -0.6094, -0.5207, -0.3959, -0.3635, -0.1697, -0.1013,  0.3166,  0.2554,
         0.3487,  0.3890,  0.4465,  0.5124,  0.5385,  0.5994,  0.6664,  0.6227,
         0.6669,  0.5258,  0.3331,  0.0184, -0.2075,  0.0764, -0.1810, -0.5246,
        -1.0218, -1.3572, -1.5903, -1.6416, -1.3137, -0.8695, -0.0934,  0.7663,
         1.0960,  0.5982,  0.5164,  0.5974,  0.9842,  0.8740,  0.7538,  0.7229,
         0.6756,  0.5636,  0.4246,  0.3059,  0.3062,  0.2083,  0.1757,  0.0142,
        -0.1386, -0.2314, -0.4394, -0.1810,  0.1546, -0.0461, -0.3356, -0.4749,
        -0.6408, -0.7357, -0.6148, -0.6253, -0.6563, -0.6311, -0.5412, -0.5573,
        -0.4698, -0.3013,  0.2397,  0.2861,  0.3796,  0.4887,  0.5392,  0.5882,
         0.6358,  0.5789,  0.6228,  0.6025,  0.6075,  0.4628,  0.2948,  0.1091,
        -0.1148,  0.1546])


/tmp/ipykernel_987590/416557686.py:10: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  difference = difference + np.sign(temp) * (np.abs(temp) > np.pi)


In [13]:
import h5py
with h5py.File("../data/UT_HAR_CAL_PHASE/X.h5","w") as f:
    f.create_dataset("X",data=test_output)